In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
#plotly
import plotly.offline as py
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from plotly import tools
init_notebook_mode(connected=True)
import plotly.figure_factory as ff
import plotly.express as px

In [ ]:
df=pd.read_csv('../input/default-of-credit-card-clients-dataset/UCI_Credit_Card.csv')
df.head().style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border-color': 'white'})

In [ ]:
df.describe().T.style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border-color': 'white'})

In [ ]:
msno.bar(df)

In [ ]:
dat=ff.create_table(df.head(10))
py.iplot(dat)

In [ ]:
plt.rcParams['figure.figsize'] = (9, 9)
labels=['Yes', 'No']
plt.pie(df['default.payment.next.month'].value_counts(), explode=(0,0.1), labels=labels,autopct='%1.1f%%', shadow=True)
plt.title('Default Payement ', fontsize = 20)
plt.axis('off')
plt.legend()
plt.show()

In [ ]:
col = "default.payment.next.month"
grouped = df[col].value_counts().reset_index()
grouped = grouped.rename(columns = {col : "count", "index" : col})

## plot
trace = go.Pie(labels=grouped[col], values=grouped['count'], pull=[0.05, 0], marker=dict(colors=["#6ad49b", "#a678de"]))
layout = go.Layout(title="", height=600, legend=dict(x=0.1, y=1.1))
fig = go.Figure(data = [trace], layout = layout)
iplot(fig)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, classification_report, confusion_matrix
from sklearn.metrics import roc_curve, precision_recall_curve

In [ ]:
X=df.drop(['default.payment.next.month'], axis=1)
y=df['default.payment.next.month']

# Spliting the dataset into train and test model

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X , y, test_size=0.3, random_state=7)


In [ ]:
lr=LogisticRegression()
lr.fit(X,y)

In [ ]:
y_pred=lr.predict(X_train)


In [ ]:
print("Accuracy Score:-", metrics.accuracy_score(y_train, y_pred))
print("F1 Score:-", metrics.f1_score(y_train, y_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_train, y_pred))
print("Log Loss:-", metrics.log_loss(y_train, y_pred))
print("Precision Score:-", metrics.precision_score(y_train, y_pred))
print("Recall Score:-", metrics.recall_score(y_train, y_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_train, y_pred))

In [ ]:
# Predicting the Test data with model 
y_test_pred=lr.predict(X_test)

In [ ]:
lr_acc=metrics.accuracy_score(y_test, y_test_pred)
print("Accuracy Score:-",lr_acc)
print("F1 Score:-", metrics.f1_score(y_test, y_test_pred))
print("Average Precision Score:-", metrics.average_precision_score(y_test, y_test_pred))
print("Log Loss:-", metrics.log_loss(y_test, y_test_pred))
print("Precision Score:-", metrics.precision_score(y_test, y_test_pred))
print("Recall Score:-", metrics.recall_score(y_test, y_test_pred))
print("ROC-AUC Score:-", metrics.roc_auc_score(y_test, y_test_pred))

In [ ]:
cfm=confusion_matrix(y_test, y_test_pred)
trueNegative=cfm[0][0]
falsePossitive=cfm[0][1]
false_negative=cfm[1][0]
truePositive=cfm[1][1]

print("Confusion Matrix", cfm)
print("true negative", trueNegative)
print("False Positive", falsePossitive)
print("false Negative", false_negative)
print("True Positive", truePositive)

In [ ]:
print("correct prediction", 
      round((trueNegative+truePositive)/len(y_test_pred)*100, 1),'%')

In [ ]:
cfm_df=pd.DataFrame(cfm, range(2), range(2))
plt.figure(figsize=(10,10))
sns.heatmap(cfm_df, cmap='Reds', annot=True)
plt.show()

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cfm,display_labels=lr.classes_)

disp.plot()

In [ ]:
print(classification_report(y_test, y_test_pred))


In [ ]:
y_test_pred_prob=lr.predict_proba(X_test)[:,1]


In [ ]:
y_test_pred_prob


In [ ]:
metrics.roc_auc_score(y_test, y_test_pred_prob)


In [ ]:
fpr, tpr,thresholds=roc_curve(y_test,y_test_pred_prob)


In [ ]:
plt.figure(figsize=(15,10),facecolor='grey')
plt.plot([0,1],[0,1],'k--')
ax = plt.axes()
  
# Setting the background color of the plot 
# using set_facecolor() method
ax.set_facecolor("grey")
plt.plot(fpr, tpr, label='Logistic Regression',linestyle='--')
plt.xlabel("fpr (False Possitive rate)")
plt.ylabel("tpr-(True Positive rate)")
plt.title("ROC_AUC")
plt.show()

In [ ]:
plt.figure(figsize=(15,10),facecolor='grey')
no_skill=len(y==1)/len(y)
y_test_prob=lr.predict_proba(X_test)[:,1]
plt.figure(figsize=(10,8))
ax = plt.axes()
  
# Setting the background color of the plot 
# using set_facecolor() method
ax.set_facecolor("grey")
plt.plot([0,1],[no_skill, no_skill], label="No Skill")
precision, recall,_ =precision_recall_curve(y_test, y_test_prob)
plt.plot(recall, precision, marker='',label="Logistic Regression")

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Recall-Precision Curve")
plt.legend()
plt.grid()

plt.show()

In [ ]:
from sklearn.metrics import log_loss
LogLoss = log_loss(y_train, y_pred, eps = 1e-15,
    normalize = True, sample_weight = None, labels = None)

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf


In [ ]:
df.columns

In [ ]:
df=df.rename(columns={'default.payment.next.month':'default'})

In [ ]:
df.columns

In [ ]:
formula='default ~ LIMIT_BAL+SEX+EDUCATION+PAY_0+PAY_2+PAY_3+PAY_4+PAY_5+PAY_6+BILL_AMT1+BILL_AMT2+BILL_AMT3+BILL_AMT4+BILL_AMT5+BILL_AMT6'

# glm()
* The glm() function fits generalized linear models, a class of models that includes logistic regression. The syntax of the glm() function is similar to that of lm(), except that we must pass in the argument family=sm.families.Binomial() in order to tell python to run a logistic regression rather than some other type of generalized linear model.

In [ ]:
model = smf.glm(formula = formula, data=df, family=sm.families.Binomial())
result = model.fit()
print(result.summary())

* The predict() function can be used to predict the probability that the market will go down, given values of the predictors. If no data set is supplied to the predict() function, then the probabilities are computed for the training data that was used to fit the logistic regression model.

In [ ]:
predictions = result.predict()
print(predictions[0:10])

In [ ]:
prediction=pd.DataFrame(predictions)

In [ ]:
prediction.head(100).plot()